# Setup Data for Homework

### <span style="color:red">IMPORTANT: Only modify cells which have the following comment:</span>
```python
# Modify this cell
```
##### <span style="color:red">Do not add any new cells when you submit the homework</span>

First we import necessary files and create the spark context. Then we load in the dataset we will be working with.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

In [3]:
import numpy as np
import Tester.moreRDD as moreRDD
pickleFile="Tester/moreRDD.pkl"

In [4]:
import Tester.moreRDD as moreRDD
data = moreRDD.getData(sc)

In [5]:
print "Python Code for getData():"
import inspect
for i in inspect.getsourcelines(moreRDD.getData)[0]:
    print i[:-1]
print
print "Number of Variables: "+str( data.count() )
print "\nVariable Exampe:"
print data.take(1)

Python Code for getData():
def getData(sc):
    f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")
    data_file = "./kddcup.data_10_percent.gz"
    return sc.textFile(data_file)

Number of Variables: 494021

Variable Exampe:
[u'0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.']


# Exercise 1
Write a function **csv_to_kv** to map each row in `data` to a key-value pair where the key is the last element in the row (network interaction type) and the value is a list of all of the elements in the row. Return the resulting RDD.

######  <span style="color:blue">Code:</span>
```python
print csv_to_kv(data).take(1)
```
######  <span style="color:magenta">Output:</span>
```python
[(u'normal.', [u'0', u'tcp', u'http', u'SF', u'181', u'5450', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'8', u'8', u'0.00', u'0.00', u'0.00', u'0.00', u'1.00', u'0.00', u'0.00', u'9', u'9', u'1.00', u'0.00', u'0.11', u'0.00', u'0.00', u'0.00', u'0.00', u'0.00', u'normal.'])]
```

In [6]:
# Modify this cell
def csv_to_kv(data):
    KV = data.map(lambda x: x.split(","))\
        .map(lambda x: (x[-1], x))
    return KV

In [7]:
import Tester.moreRDD as moreRDD
moreRDD.exercise1(pickleFile, csv_to_kv, data ,sc)

Input: <class 'pyspark.rdd.RDD'>
Correct Output: [(u'normal.', [u'0', u'tcp', u'http', u'SF', u'181', u'5450', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'8', u'8', u'0.00', u'0.00', u'0.00', u'0.00', u'1.00', u'0.00', u'0.00', u'9', u'9', u'1.00', u'0.00', u'0.11', u'0.00', u'0.00', u'0.00', u'0.00', u'0.00', u'normal.']), (u'normal.', [u'0', u'tcp', u'http', u'SF', u'239', u'486', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'8', u'8', u'0.00', u'0.00', u'0.00', u'0.00', u'1.00', u'0.00', u'0.00', u'19', u'19', u'1.00', u'0.00', u'0.05', u'0.00', u'0.00', u'0.00', u'0.00', u'0.00', u'normal.']), (u'normal.', [u'0', u'tcp', u'http', u'SF', u'235', u'1337', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'8', u'8', u'0.00', u'0.00', u'0.00', u'0.00', u'1.00', u'0.00', u'0.00', u'29', u'29', u'1.00', u'0.00', u'0.03', u'0.00', u'0.00'

# Exercise 2
Create a new function **orderNet**, that returns an RDD of the sorted network interaction types by total duration. Duration is the first column of x, i.e. x[0]. Your RDD should contain tuples of the form: (interaction_type, total_duration).

    Note: You will want to use csv_to_kv from exercise 1 

######  <span style="color:blue">Code:</span>
```python
print orderNet(data).take(5)

```
######  <span style="color:magenta">Output:</span>
`
[(u'normal.', 21075991), (u'portsweep.', 1991911), (u'warezclient.', 627563), (u'buffer_overflow.', 2751), (u'multihop.', 1288)]
`

In [8]:
# Modify this cell
def orderNet(data):
    RDD = csv_to_kv(data)  #<-- The correct RDD from csv_to_kv(data)
    orderedRDD = RDD.map(lambda x: (x[0], int(x[1][0])))\
        .reduceByKey(lambda x,y: x+y)\
        .map(lambda (k,v): (v, k))\
        .sortByKey(False)\
        .map(lambda (v,k): (k,v))
    return orderedRDD

In [9]:
import Tester.moreRDD as moreRDD
moreRDD.exercise2(pickleFile, orderNet, data ,sc)

Input: <class 'pyspark.rdd.RDD'>
Correct Output: [(u'normal.', 21075991), (u'portsweep.', 1991911), (u'warezclient.', 627563), (u'buffer_overflow.', 2751), (u'multihop.', 1288)]
Great Job!


# Exercise 3

Continue with the functions created in previous exercises. Create a function, **avgDuration**, that calculates and returns the average duration for each of the network interaction types. Order the data by key name. You are encouraged to use **combineByKey()**.

    Note: You will want to use functions from the previous exercises


######  <span style="color:blue">Code:</span>
```python
print avgDuration(data).take(9)
```
######  <span style="color:magenta">Output:</span>
`
[(u'back.', 0.1289151157512483), (u'buffer_overflow.', 91.7), (u'ftp_write.', 32.375), (u'guess_passwd.', 2.7169811320754715), (u'imap.', 6.0), (u'ipsweep.', 0.034482758620689655), (u'land.', 0.0), (u'loadmodule.', 36.22222222222222), (u'multihop.', 184.0)]
`

In [10]:
# Modify this cell
def avgDuration(data):
    RDD1 = csv_to_kv(data)  #<-- The correct RDD from csv_to_kv(data)
    RDD = RDD1.map(lambda x: (x[0], int(x[1][0])))
    RDD2 = orderNet(data)   #<-- The correct RDD from orderNet(data)
    
    sumCount = RDD.combineByKey(lambda value: (value, 1),
                                lambda x, value: (x[0] + value, x[1] + 1),
                                lambda x, y: (x[0] + y[0], x[1] + y[1]))
    
    avgRDD = sumCount.map(lambda (label, (value_sum, count)): (label, value_sum*1.0 / count))\
        .sortBy(ascending=1, keyfunc=lambda x: x[0])
    return avgRDD

In [11]:
import Tester.moreRDD as moreRDD
moreRDD.exercise3(pickleFile, avgDuration, data ,sc)

Input: <class 'pyspark.rdd.RDD'>
Correct Output: [(u'back.', 0.1289151157512483), (u'buffer_overflow.', 91.7), (u'ftp_write.', 32.375), (u'guess_passwd.', 2.7169811320754715), (u'imap.', 6.0), (u'ipsweep.', 0.034482758620689655), (u'land.', 0.0), (u'loadmodule.', 36.22222222222222), (u'multihop.', 184.0)]
Great Job!


True